# Task 3 – Hypothesis Testing: Insurance Risk & Margin

This notebook performs statistical hypothesis testing for AlphaCare Insurance Solutions (ACIS).

We focus on four null hypotheses:

1. **H₀₁:** There are no risk differences across provinces.  
2. **H₀₂:** There are no risk differences between zip codes.  
3. **H₀₃:** There is no significant margin (profit) difference between zip codes.  
4. **H₀₄:** There is no significant risk difference between Women and Men.

**Risk metrics:**

- **Claim Frequency:** proportion of policies with at least one claim (`TotalClaims > 0`).  
- **Claim Severity:** average claim amount, conditional on `TotalClaims > 0`.  

**Margin:**

- `margin = TotalPremium - TotalClaims`.

We use:

- Chi-square tests for categorical vs binary outcomes (e.g., Province vs has_claim).  
- t-tests / ANOVA for continuous outcomes (e.g., severity, margin).
- A significance level of α = 0.05.


In [19]:
import pandas as pd
import numpy as np

from scipy.stats import chi2_contingency, ttest_ind, f_oneway

pd.set_option("display.max_rows", 50)
pd.set_option("display.max_columns", 50)

DATA_PATH = "../data/raw/MachineLearningRating_v3.txt"

df = pd.read_csv(DATA_PATH, sep="|")

df.head()


C:\Users\Dawit\AppData\Local\Temp\ipykernel_6728\4102506430.py:11: DtypeWarning: Columns (32,37) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(DATA_PATH, sep="|")


,UnderwrittenCoverID,PolicyID,TransactionMonth,IsVATRegistered,Citizenship,LegalType,Title,Language,Bank,AccountType,MaritalStatus,Gender,Country,Province,PostalCode,MainCrestaZone,SubCrestaZone,ItemType,mmcode,VehicleType,RegistrationYear,make,Model,Cylinders,cubiccapacity,...,NumberOfDoors,VehicleIntroDate,CustomValueEstimate,AlarmImmobiliser,TrackingDevice,CapitalOutstanding,NewVehicle,WrittenOff,Rebuilt,Converted,CrossBorder,NumberOfVehiclesInFleet,SumInsured,TermFrequency,CalculatedPremiumPerTerm,ExcessSelected,CoverCategory,CoverType,CoverGroup,Section,Product,StatutoryClass,StatutoryRiskType,TotalPremium,TotalClaims
0,145249,12827,2015-03-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,Not specified,Not specified,South Africa,Gauteng,1459,Rand East,Rand East,Mobility - Motor,44069150.0,Passenger Vehicle,2004,MERCEDES-BENZ,E 240,6.0,2597.0,...,4.0,6/2002,119300.0,Yes,No,119300,More than 6 months,NaN,NaN,NaN,NaN,NaN,0.01,Monthly,25.0000,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,21.929825,0.0
1,145249,12827,2015-05-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,Not specified,Not specified,South Africa,Gauteng,1459,Rand East,Rand East,Mobility - Motor,44069150.0,Passenger Vehicle,2004,MERCEDES-BENZ,E 240,6.0,2597.0,...,4.0,6/2002,119300.0,Yes,No,119300,More than 6 months,NaN,NaN,NaN,NaN,NaN,0.01,Monthly,25.0000,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,21.929825,0.0
2,145249,12827,2015-07-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,Not specified,Not specified,South Africa,Gauteng,1459,Rand East,Rand East,Mobility - Motor,44069150.0,Passenger Vehicle,2004,MERCEDES-BENZ,E 240,6.0,2597.0,...,4.0,6/2002,119300.0,Yes,No,119300,More than 6 months,NaN,NaN,NaN,NaN,NaN,0.01,Monthly,25.0000,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,0.000000,0.0
3,145255,12827,2015-05-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,Not specified,Not specified,South Africa,Gauteng,1459,Rand East,Rand East,Mobility - Motor,44069150.0,Passenger Vehicle,2004,MERCEDES-BENZ,E 240,6.0,2597.0,...,4.0,6/2002,119300.0,Yes,No,119300,More than 6 months,NaN,NaN,NaN,NaN,NaN,119300.00,Monthly,584.6468,Mobility - Metered Taxis - R2000,Own damage,Own Damage,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,512.848070,0.0
4,145255,12827,2015-07-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,Not specified,Not specified,South Africa,Gauteng,1459,Rand East,Rand East,Mobility - Motor,44069150.0,Passenger Vehicle,2004,MERCEDES-BENZ,E 240,6.0,2597.0,...,4.0,6/2002,119300.0,Yes,No,119300,More than 6 months,NaN,NaN,NaN,NaN,NaN,119300.00,Monthly,584.6468,Mobility - Metered Taxis - R2000,Own damage,Own Damage,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,0.000000,0.0


## 1. Basic Preparation and Derived Metrics

We create:

- `has_claim` – indicator of whether a policy has any claims.  
- `margin` – `TotalPremium - TotalClaims`.  
- `loss_ratio` – `TotalClaims / TotalPremium` when premium > 0.  

We also do light type-cleaning for key fields used in tests.


In [20]:
# Ensure numeric columns are numeric (just in case)
for col in ["TotalPremium", "TotalClaims", "SumInsured", "CalculatedPremiumPerTerm"]:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce")

# Claim indicator (frequency)
df["has_claim"] = (df["TotalClaims"] > 0).astype(int)

# Margin = TotalPremium - TotalClaims
df["margin"] = df["TotalPremium"] - df["TotalClaims"]

# Loss ratio, guarding against division by zero
df["loss_ratio"] = np.where(
    df["TotalPremium"] > 0,
    df["TotalClaims"] / df["TotalPremium"],
    np.nan
)

# Subset with claims only for severity analyses
df_claims = df[df["TotalClaims"] > 0].copy()

df[["TotalPremium", "TotalClaims", "has_claim", "margin", "loss_ratio"]].describe()


,TotalPremium,TotalClaims,has_claim,margin,loss_ratio
count,1.000098e+06,1.000098e+06,1.000098e+06,1.000098e+06,618176.000000
mean,6.190550e+01,6.486119e+01,2.787727e-03,-2.955694e+00,0.350048
std,2.302845e+02,2.384075e+03,5.272531e-02,2.367137e+03,9.288639
min,-7.825768e+02,-1.200241e+04,0.000000e+00,-3.928486e+05,-18.700122
25%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000
50%,2.178333e+00,0.000000e+00,0.000000e+00,2.157687e+00,0.000000
75%,2.192982e+01,0.000000e+00,0.000000e+00,2.192982e+01,0.000000
max,6.528260e+04,3.930921e+05,1.000000e+00,6.528260e+04,2553.600000


In [21]:
def print_test_result(test_name: str, h0_description: str, p_value: float, alpha: float = 0.05):
    """
    Pretty-print the results of a hypothesis test.
    """
    print(f"=== {test_name} ===")
    print(f"H0: {h0_description}")
    print(f"p-value = {p_value:.4g}")
    if p_value < alpha:
        print(f"Decision: Reject H0 at α={alpha}")
    else:
        print(f"Decision: Fail to reject H0 at α={alpha}")
    print("-" * 60)


## 2. Hypothesis 1 – Risk Differences Across Provinces

**H₀₁:** There are no risk differences across provinces.

We test:

1. **Claim Frequency vs Province** – Chi-square test of independence.  
2. **Claim Severity vs Province** – One-way ANOVA on `TotalClaims` for records with claims.


In [22]:
# Drop records with missing Province
df_prov = df.dropna(subset=["Province"]).copy()

# Summary of exposure and risk by Province
province_summary = (
    df_prov.groupby("Province")
    .agg(
        policies=("UnderwrittenCoverID", "count"),
        claim_frequency=("has_claim", "mean"),
        avg_severity=("TotalClaims", lambda x: x[x > 0].mean()),
        avg_loss_ratio=("loss_ratio", "mean"),
        avg_margin=("margin", "mean")
    )
    .sort_values("policies", ascending=False)
)

province_summary.head(10)


,policies,claim_frequency,avg_severity,avg_loss_ratio,avg_margin
Province,,,,,
Gauteng,393865,0.003356,22243.878396,0.428927,-13.558894
Western Cape,170796,0.002166,28095.849881,0.341810,-3.414689
KwaZulu-Natal,169781,0.002845,29609.487473,0.265328,-6.433598
North West,143287,0.002436,16963.467035,0.285348,10.958832
Mpumalanga,52718,0.002428,15979.553421,0.392698,15.016059
Eastern Cape,30336,0.001648,27128.533277,0.235636,25.833240
Limpopo,24836,0.002698,15171.294187,0.348712,20.971484
Free State,8099,0.001358,32265.661085,0.106209,20.550805
Northern Cape,6380,0.001254,11186.313596,0.203831,35.590527


In [23]:
# Contingency table: Province x has_claim (0/1)
cont_prov_freq = pd.crosstab(df_prov["Province"], df_prov["has_claim"])

chi2, p_freq_prov, dof, expected = chi2_contingency(cont_prov_freq)

print_test_result(
    test_name="Chi-square test: Claim frequency vs Province",
    h0_description="Claim frequency is independent of Province (no risk differences)",
    p_value=p_freq_prov
)


=== Chi-square test: Claim frequency vs Province ===
H0: Claim frequency is independent of Province (no risk differences)
p-value = 5.926e-19
Decision: Reject H0 at α=0.05
------------------------------------------------------------


In [24]:
df_claims_prov = df_claims.dropna(subset=["Province"]).copy()

# Build list of severity samples per province (filter out very tiny groups)
severity_groups_prov = [
    group["TotalClaims"].values
    for _, group in df_claims_prov.groupby("Province")
    if len(group) > 30  # avoid tiny groups for ANOVA
]

f_stat_prov, p_sev_prov = f_oneway(*severity_groups_prov)

print_test_result(
    test_name="ANOVA: Claim severity vs Province (claims only)",
    h0_description="Mean claim severity is equal across provinces",
    p_value=p_sev_prov
)


=== ANOVA: Claim severity vs Province (claims only) ===
H0: Mean claim severity is equal across provinces
p-value = 1.744e-06
Decision: Reject H0 at α=0.05
------------------------------------------------------------


### Interpretation – Provinces

Use the `province_summary` table and the p-values above to describe:

- Whether claim frequency differs significantly across provinces.
- Whether claim severity differs significantly across provinces.
- Which provinces appear higher or lower risk based on frequency, severity, and loss ratio.

You will use this in your written report to recommend whether **provincial rating factors** are justified.


## 3. Hypothesis 2 – Risk Differences Between Zip Codes

**H₀₂:** There are no risk differences between zip codes.

Because there are many zip codes, we focus on the **top N zip codes** by exposure (policy count).  
We again test:

1. Claim frequency vs PostalCode (chi-square).  
2. Claim severity vs PostalCode (ANOVA, claims only).


In [25]:
# Drop missing PostalCode, and coerce to integer
df_zip = df.dropna(subset=["PostalCode"]).copy()
df_zip["PostalCode"] = df_zip["PostalCode"].astype(int)

# Top N zip codes by number of policies
top_n = 10
top_zips = df_zip["PostalCode"].value_counts().head(top_n).index

df_top_zip = df_zip[df_zip["PostalCode"].isin(top_zips)].copy()

# Summary table for the selected zip codes
zip_risk_summary = (
    df_top_zip.groupby("PostalCode")
    .agg(
        policies=("UnderwrittenCoverID", "count"),
        claim_frequency=("has_claim", "mean"),
        avg_severity=("TotalClaims", lambda x: x[x > 0].mean()),
        avg_loss_ratio=("loss_ratio", "mean"),
        avg_margin=("margin", "mean")
    )
    .sort_values("policies", ascending=False)
)

zip_risk_summary


,policies,claim_frequency,avg_severity,avg_loss_ratio,avg_margin
PostalCode,,,,,
2000,133498,0.003641,19196.413727,0.414397,-8.111944
122,49171,0.004271,18162.025865,0.441108,-22.859806
7784,28585,0.001749,35156.653709,0.308058,-13.471242
299,25546,0.002623,13622.745632,0.223981,19.559415
7405,18518,0.001566,21002.022686,0.176914,17.516825
458,13775,0.002323,20160.273246,0.350525,4.546074
8000,11794,0.004324,33685.329976,0.718501,-16.036381
2196,11048,0.002896,50877.777961,0.734474,-21.341082
470,10226,0.004303,12946.833234,0.436997,3.069148


In [26]:
cont_zip_freq = pd.crosstab(df_top_zip["PostalCode"], df_top_zip["has_claim"])

chi2_zip, p_freq_zip, dof_zip, exp_zip = chi2_contingency(cont_zip_freq)

print_test_result(
    test_name=f"Chi-square: Claim frequency across top {top_n} zip codes",
    h0_description="Claim frequency is equal across selected zip codes",
    p_value=p_freq_zip
)


=== Chi-square: Claim frequency across top 10 zip codes ===
H0: Claim frequency is equal across selected zip codes
p-value = 4.593e-12
Decision: Reject H0 at α=0.05
------------------------------------------------------------


In [27]:
df_claims_top_zip = df_claims.dropna(subset=["PostalCode"]).copy()
df_claims_top_zip["PostalCode"] = df_claims_top_zip["PostalCode"].astype(int)
df_claims_top_zip = df_claims_top_zip[df_claims_top_zip["PostalCode"].isin(top_zips)]

severity_groups_zip = [
    group["TotalClaims"].values
    for _, group in df_claims_top_zip.groupby("PostalCode")
    if len(group) > 30
]

f_zip, p_sev_zip = f_oneway(*severity_groups_zip)

print_test_result(
    test_name=f"ANOVA: Claim severity across top {top_n} zip codes (claims only)",
    h0_description="Mean claim severity is equal across selected zip codes",
    p_value=p_sev_zip
)


=== ANOVA: Claim severity across top 10 zip codes (claims only) ===
H0: Mean claim severity is equal across selected zip codes
p-value = 1.766e-07
Decision: Reject H0 at α=0.05
------------------------------------------------------------


### Interpretation – Zip Codes

Using `zip_risk_summary` and the test results:

- Comment on which zip codes have **higher claim frequency**, **higher severity**, or **worse loss ratio**.
- If H₀₂ is rejected, explain that zip code is a meaningful risk segment.
- Relate this to potential **micro-geographic pricing** (higher premiums for high-risk zip codes, potential discounts for low-risk pockets).


## 4. Hypothesis 3 – Margin Differences Between Zip Codes

**H₀₃:** There is no significant margin (profit) difference between zip codes.

Here, we test whether the **mean margin** differs across the same top N zip codes, using one-way ANOVA.


In [28]:
# Use same df_top_zip, but keep only records with positive premium for sensible margin
df_margin_top = df_top_zip[df_top_zip["TotalPremium"] > 0].copy()

margin_groups_zip = [
    group["margin"].values
    for _, group in df_margin_top.groupby("PostalCode")
    if len(group) > 30
]

f_margin_zip, p_margin_zip = f_oneway(*margin_groups_zip)

print_test_result(
    test_name=f"ANOVA: Margin across top {top_n} zip codes",
    h0_description="Mean margin is equal across selected zip codes",
    p_value=p_margin_zip
)

# Margin summary table for interpretation
zip_margin_summary = (
    df_margin_top.groupby("PostalCode")
    .agg(
        policies=("UnderwrittenCoverID", "count"),
        avg_margin=("margin", "mean"),
        avg_loss_ratio=("loss_ratio", "mean"),
        claim_frequency=("has_claim", "mean")
    )
    .sort_values("avg_margin", ascending=False)
)

zip_margin_summary


=== ANOVA: Margin across top 10 zip codes ===
H0: Mean margin is equal across selected zip codes
p-value = 0.4699
Decision: Fail to reject H0 at α=0.05
------------------------------------------------------------


,policies,avg_margin,avg_loss_ratio,claim_frequency
PostalCode,,,,
299,16731,34.843901,0.223981,0.003885
7405,10731,30.227990,0.176914,0.002702
470,7052,23.464405,0.436997,0.005814
7100,5906,16.430814,0.237464,0.004402
458,8392,7.598012,0.350525,0.003694
2000,90934,-8.939661,0.414397,0.005180
7784,13560,-20.787465,0.308058,0.003319
8000,8855,-21.079078,0.718501,0.005534
122,27898,-22.297306,0.441108,0.006811


### Interpretation – Margin by Zip Code

From `zip_margin_summary` and the p-value:

- If we **reject H₀₃**, we conclude that profitability (margin) is not uniform across zip codes.
- High-margin zip codes may present opportunities for **targeted discounts** or customer acquisition.
- Low-margin or negative-margin zip codes may require **premium increases**, stricter underwriting, or further investigation into fraud/claims patterns.


## 5. Hypothesis 4 – Risk Differences Between Women and Men

**H₀₄:** There is no significant risk difference between Women and Men.

We compare:

1. Claim frequency between Women and Men (chi-square).  
2. Claim severity between Women and Men (t-test on `TotalClaims` for records with claims).

First, we clean the `Gender` column and focus on well-defined `"Female"` and `"Male"` categories.


In [29]:
df["Gender"].value_counts(dropna=False)


Gender
Not specified    940990
Male              42817
NaN                9536
Female             6755
Name: count, dtype: int64

In [30]:
# Adjust these labels based on the value_counts output
female_label = "Female"
male_label = "Male"

df_gender = df[df["Gender"].isin([female_label, male_label])].copy()
df_gender["has_claim"] = (df_gender["TotalClaims"] > 0).astype(int)

gender_summary = (
    df_gender.groupby("Gender")
    .agg(
        policies=("UnderwrittenCoverID", "count"),
        claim_frequency=("has_claim", "mean"),
        avg_severity=("TotalClaims", lambda x: x[x > 0].mean()),
        avg_loss_ratio=("loss_ratio", "mean"),
        avg_margin=("margin", "mean")
    )
)

gender_summary


,policies,claim_frequency,avg_severity,avg_loss_ratio,avg_margin
Gender,,,,,
Female,6755,0.002073,17874.721303,0.491979,8.028787
Male,42817,0.002195,14858.552294,0.353258,4.284253


In [31]:
cont_gender_freq = pd.crosstab(df_gender["Gender"], df_gender["has_claim"])

chi2_gender, p_freq_gender, dof_gender, exp_gender = chi2_contingency(cont_gender_freq)

print_test_result(
    test_name="Chi-square: Claim frequency vs Gender",
    h0_description="Claim frequency is equal for Women and Men",
    p_value=p_freq_gender
)


=== Chi-square: Claim frequency vs Gender ===
H0: Claim frequency is equal for Women and Men
p-value = 0.9515
Decision: Fail to reject H0 at α=0.05
------------------------------------------------------------


In [32]:
df_gender_claims = df_claims[df_claims["Gender"].isin([female_label, male_label])].copy()

claims_female = df_gender_claims[df_gender_claims["Gender"] == female_label]["TotalClaims"]
claims_male   = df_gender_claims[df_gender_claims["Gender"] == male_label]["TotalClaims"]

t_stat_gender, p_sev_gender = ttest_ind(
    claims_female,
    claims_male,
    equal_var=False  # Welch's t-test (robust to unequal variances)
)

print_test_result(
    test_name="T-test: Claim severity vs Gender (claims only)",
    h0_description="Mean claim severity is equal for Women and Men",
    p_value=p_sev_gender
)


=== T-test: Claim severity vs Gender (claims only) ===
H0: Mean claim severity is equal for Women and Men
p-value = 0.568
Decision: Fail to reject H0 at α=0.05
------------------------------------------------------------


### Interpretation – Gender

Using `gender_summary` and the test results:

- Comment on whether Women and Men show different **claim frequencies** and/or **claim severities**.
- If H₀₄ is rejected, give a business explanation such as:

> "We find statistically significant differences in risk between women and men. For example, women show lower claim frequency and slightly lower severity, suggesting that gender (subject to regulatory constraints) could be a useful risk factor in pricing or underwriting."

Always check local regulations on whether gender-based pricing is permitted.


## 6. Summary: Decisions and Business Recommendations

In this section we summarise, for each hypothesis (H₀₁–H₀₄):

- The metric(s) used (claim frequency, claim severity, margin).
- The statistical test applied.
- The p-value and decision (reject / fail to reject).
- The business interpretation based on group-level summaries.

### Provinces (H₀₁)

**Result:** We reject the null hypothesis that risk is equal across provinces (χ² and ANOVA both p < 0.001). Gauteng, for example, shows higher claim frequency and a worse average loss ratio (≈0.43) with negative average margin, while provinces such as Northern Cape and Eastern Cape have lower loss ratios (≈0.20–0.24) and positive margins. This supports introducing **province-based rating factors** and closer review of underwriting and pricing in high-risk provinces such as Gauteng.

### Zip Codes – Risk (H₀₂)

**Result:** For the top 10 zip codes by exposure, we find significant differences in both claim frequency and claim severity (χ² and ANOVA p < 0.001). This indicates that **micro-geographic segmentation at zip-code level** can improve risk differentiation, enabling higher premiums or stricter terms in high-claim areas and potential discounts in consistently better-performing zip codes.

### Zip Codes – Margin (H₀₃)

**Result:** The ANOVA on margin across the top 10 zip codes yields p ≈ 0.47, so we **fail to reject** the null hypothesis that mean margin is equal across these zip codes. Although point estimates of average margin differ (some zip codes positive, others negative), the variability within each group is large enough that we do not have strong statistical evidence of systematic profitability differences at this level. Zip-based segmentation is still useful for risk, but margin alone does not yet justify aggressive profit rebalancing.

### Gender (H₀₄)

**Result:** We do not find statistically significant differences in risk between women and men. The chi-square test for claim frequency (p ≈ 0.95) and the Welch t-test for claim severity among claimants (p ≈ 0.57) both lead us to **fail to reject** the null hypothesis. Combined with the fact that most records have “Not specified” as gender, this suggests that **gender is not a reliable or material risk driver** in this portfolio. Other features such as geography, vehicle characteristics and cover type are more relevant levers for pricing and segmentation, subject to regulatory constraints.

These statistical decisions and interpretations directly inform ACIS’s marketing and pricing strategy by highlighting **where** risk differences are strongest (province, zip code) and **which variables** are less useful (gender, zip-level margin) for immediate pricing action.
